# 17. Both, BothE, BothV, OtherV
#### By Kwan Yin Andrew Chau

In [1]:
import nbfinder
from loader import g
from loader import __
from loader import Barrier
from loader import Column
from loader import P
from loader import Pick
from loader import Pop
from loader import T
from loader import sydneyVertexId
from loader import hkVertexId

## sideEffect()

__Store destinations directly out of HK and destinations 2 stops away__

Side effects do not change what gets passed on to the next stage of the query

In [2]:
g.V(hkVertexId).sideEffect(
    __.out().count().store('direct')
).out().out().count().as_('twoStopsAway').select('direct','twoStopsAway').next()

{'direct': [151], 'twoStopsAway': 11716}

__This can also be done by grouping into a map__

In [3]:
runningSe = lambda: print("running...")
# TODO
#g.V().has('code','SFO').sideEffect(runningSe).next()

Count how many places we can go to from HK within one stop

In [4]:
x1 = g.V(hkVertexId).out().out().dedup().count().next()
x1

1883

## aggregate()

__Aggregate allows us to create a temporary collection__

Count how many places we can go to from HK that is only possible when stopping once

In [5]:
x2 = g.V(hkVertexId).out().aggregate('direct').out().where(P.without('direct')).dedup().count().next()
x2

1732

This is equal to the non-aggregate count minus the direct count

In [6]:
directCount = g.V(hkVertexId).out().count().next()
assert x1 - directCount == x2

## inject()

__Inject the rating into the values of the HK vertex__

In [7]:
g.V(hkVertexId).values().inject(8).inject('Rating').fold().next()

['Rating',
 8,
 'HKG',
 'airport',
 'Hong Kong - Chek Lap Kok International Airport',
 'HK',
 12467,
 'Hong Kong',
 28,
 'VHHH',
 113.915000916,
 'HK',
 2,
 22.3089008331]

__Look for whether there is the ABC vertex__

This however returns a result for every vertex traversed

In [8]:
g.V().choose(__.V("ABC").count().is_(0), __.constant("None found")).fold().next()

['None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None found',
 'None fou

We can limit this down to how ever many by injecting things after the source (g)

In [9]:
g.inject("a", "b").choose(__.V("ABC").count().is_(0), __.constant("None found")).fold().next()

['None found', 'None found']

## coalesce()
__Use the first step inside coalesce that returns at least once vertex__

In [10]:
g.V(hkVertexId).coalesce(__.out('fake'), __.in_('contains')).values('desc').fold().next()

['Hong Kong', 'Asia']

constant() can be used as a fallback to coalesce()

In [11]:
g.V(hkVertexId).coalesce(__.out('fake'), __.out('fake2'), __.constant('No Match')).next()

'No Match'

## optional()
__Return the prior step (HK) if there is no route to a certain code__

In [12]:
x1 = g.V(hkVertexId).optional(__.out().has('code','FAKE')).values('city').fold().next()
x1

['Hong Kong']

In [13]:
y1 = g.V(hkVertexId).optional(__.out().hasId(sydneyVertexId)).values('city').fold().next()
y1

['Sydney']

We can do the same thing by using identity() with coalesce()

In [14]:
x2 = g.V(hkVertexId).coalesce(__.out().has('code','FAKE'), __.identity()).values('city').fold().next()
assert x1 == x2

In [15]:
y2 = g.V(hkVertexId).coalesce(__.out().hasId(sydneyVertexId), __.identity()).values('city').fold().next()
assert y1 == y2